Nama : Gebby Novalisza </br>
NIM : 1102201399 </br>
Kelas : EL-43-D

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [ ]:
import os
import gzip
import shutil

with gzip.open('/content/movie_data.csv.gz', 'rb') as f_in, open('/content/movie_data.csv', 'wb') as f_out:
  shutil.copyfileobj(f_in, f_out)


In [ ]:
df = pd.read_csv('/content/movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [ ]:
# Step 1: Create a dataset

target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices(
 (df.values, target.values))

## inspection:
for ex in ds_raw.take(3):
 tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [ ]:
# 1. Buat objek dataser TensorFlow dan pisahkan menjadi partisi training, testing, dan validasi set yang terpisah.

tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(
 50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [ ]:
## Step 2: Identifikasi kata-kata unik dalam dataset training.
from collections import Counter
try:
 tokenizer = tfds.features.text.Tokenizer()
except AttributeError:
 tokenizer = tfds.deprecated.text.Tokenizer()
 
token_counts = Counter()
for example in ds_raw_train:
 tokens = tokenizer.tokenize(example[0].numpy()[0])
 token_counts.update(tokens)
 
print('Vocab-size:', len(token_counts))

Vocab-size: 87063


In [ ]:
## Step 3: encoding each unique token into integers
try:
 encoder = tfds.features.text.TokenTextEncoder(token_counts)
except AttributeError:
 encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'This is an example!'
encoder.encode(example_str)

[544, 40, 223, 2166]

In [ ]:
## Step 3-A: define the function for transformation
def encode(text_tensor, label):
 text = text_tensor.numpy()[0]
 encoded_text = encoder.encode(text)
 return encoded_text, label

## Step 3-B: wrap the encode function to a TF Op.
def encode_map_fn(text, label):
 return tf.py_function(encode, inp=[text, label],
 Tout=(tf.int64, tf.int64))

In [ ]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)
tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
 print('Sequence length:', example[0].shape)
 
example

Sequence length: (109,)
Sequence length: (177,)
Sequence length: (281,)
Sequence length: (199,)
Sequence length: (124,)


(<tf.Tensor: shape=(124,), dtype=int64, numpy=
 array([   86,  9018, 12346, 16862,    40,    46,   653,  3385,     8,
           46,    22,    71,   181,    33, 16863,   513,    17,     9,
         3032,  2844,     8,   401, 16864, 15670,  4170,   319,   319,
          337,   299,    51,    86,   837,    40, 16224, 16865, 16866,
         2871,    44,   270,  1719,    14, 11589, 10498,   116,  1072,
         1106,   223,   313,  4994,  2488,    16,    30,   483,    58,
          268,   183,   604,   105,     9,  1842,  8139,  4170,   219,
          299,   223,  2246,   545,   515,   972,   249,    46, 16867,
           46,  3622,  3974,  9073,  4405,    46,  3032,  3142,   972,
           14,    46,  6036,    44,     9, 16868,  2046,    14,    61,
          299,    46,  8701,  1029,   139,    46,  3275,    14,    46,
        16869,   214,  4891,   319,   319,   263,  2537,   299,  1183,
          374,  4104,  3797,  8139,  4170,   299, 16870,  8519,    14,
           13,   125,    44,  

In [ ]:
## Take a small subset
ds_subset = ds_train.take(8)
for example in ds_subset:
 print('Individual size:', example[0].shape)

## batching the datasets
ds_batched = ds_subset.padded_batch(
 4, padded_shapes=([-1], []))

for batch in ds_batched:
 print('Batch dimension:', batch[0].shape)

Individual size: (236,)
Individual size: (250,)
Individual size: (193,)
Individual size: (310,)
Individual size: (143,)
Individual size: (258,)
Individual size: (214,)
Individual size: (117,)
Batch dimension: (4, 310)
Batch dimension: (4, 258)


In [ ]:
## batching the datasets
train_data = ds_train.padded_batch(
 32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
 32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
 32, padded_shapes=([-1],[]))


**Embedding Layers untuk Sentence Encoding**

In [ ]:
from tensorflow.keras.layers import Embedding

model = tf.keras.Sequential()
model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name='embed-layer'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


**Membangun RNN Model**

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 32)          2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


**Membangun RNN Model untuk Tugas Sentiment Analysis**

In [ ]:
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [68]:
embedding_dim = 20
vocab_size = len(token_counts) + 100000
tf.random.set_seed(1)
## build the model
bi_lstm_model = tf.keras.Sequential([
 tf.keras.layers.Embedding(
    input_dim=vocab_size+100000,
    output_dim=embedding_dim,
    name='embed-layer'),
 
 tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(64, name='lstm-layer'),
    name='bidir-lstm'),
 tf.keras.layers.Dense(64, activation='relu'),
 
 tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

## compile and train:

bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=2)

## evaluate on the test data
test_results= bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          5741260   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,793,101
Trainable params: 5,793,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
625/625 [==============================] - 716s 1s/step - loss: 0.5018 - accuracy: 0.7524 - val_loss: 0.3740 - val_accuracy: 0.8548
Epoch 2/2
782/782 [==============

In [ ]:
def preprocess_datasets(
 ds_raw_train,
 ds_raw_valid,
 ds_raw_test,
 max_seq_length=None,
 batch_size=32):
 
 ## Step 1: (already done => creating a dataset)
 ## Step 2: find unique tokens

  try:
    tokenizer = tfds.features.text.Tokenizer()
  except AttributeError:
    tokenizer = tfds.deprecated.text.Tokenizer()
  token_counts = Counter()

  for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    if max_seq_length is not None:
      tokens = tokens[-max_seq_length:]
    token_counts.update(tokens)

  print('Vocab-size:', len(token_counts))

  ## Step 3: encoding the texts
  try:
    encoder = tfds.features.text.TokenTextEncoder(token_counts)
  except AttributeError:
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
  def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    if max_seq_length is not None:
      encoded_text = encoded_text[-max_seq_length:]
    return encoded_text, label

    def encode_map_fn(text, label):
      return tf.py_function(encode, inp=[text, label],
                            Tout=(tf.int64, tf.int64))
    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)

  ## Step 4: batching the datasets
  train_data = ds_train.padded_batch(
      batch_size, padded_shapes=([-1],[]))
  
  valid_data = ds_valid.padded_batch(
      batch_size, padded_shapes=([-1],[]))
  
  test_data = ds_test.padded_batch(
      batch_size, padded_shapes=([-1],[]))
  
  return (train_data, valid_data,
          test_data, len(token_counts))

In [ ]:
def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):
  
  tf.random.set_seed(1)
  
  # build the model
  model = tf.keras.Sequential()
  
  model.add(
      Embedding(
          input_dim=vocab_size,
          output_dim=embedding_dim,
          name='embed-layer')
  )
 
  for i in range(n_recurrent_layers):
    return_sequences = (i < n_recurrent_layers-1)

    if recurrent_type == 'SimpleRNN':
      recurrent_layer = SimpleRNN(
          units=n_recurrent_units,
          return_sequences=return_sequences,
          name='simprnn-layer-{}'.format(i))

    elif recurrent_type == 'LSTM':
      recurrent_layer = LSTM(
          units=n_recurrent_units,
          return_sequences=return_sequences,
          name='lstm-layer-{}'.format(i))
    
    elif recurrent_type == 'GRU':
      recurrent_layer = GRU(
          units=n_recurrent_units,
          return_sequences=return_sequences,
          name='gru-layer-{}'.format(i))

    if bidirectional:
      recurrent_layer = Bidirectional(
          recurrent_layer, name='bidir-'+recurrent_layer.name)

  model.add(recurrent_layer)
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  return model

In [61]:
from tensorflow.keras.layers import Bidirectional

batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test,
    max_seq_length=max_seq_length,
    batch_size=batch_size
)

vocab_size = n + 100000

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN',
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

Vocab-size: 58112
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          3162240   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_20 (Dense)            (None, 64)                8256      
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 3,181,441
Trainable params: 3,181,441
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

history = rnn_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=2)

Epoch 1/2
625/625 [==============================] - 348s 553ms/step - loss: 0.7003 - accuracy: 0.5125 - val_loss: 0.6989 - val_accuracy: 0.5202
Epoch 2/2
625/625 [==============================] - 349s 558ms/step - loss: 0.5983 - accuracy: 0.6727 - val_loss: 0.4999 - val_accuracy: 0.7884


In [ ]:
results = rnn_model.evaluate(test_data)

782/782 [==============================] - 78s 99ms/step - loss: 0.5229 - accuracy: 0.7750


In [ ]:
print('Test Acc.: {:.2f}%'.format(results[1]*100))

Test Acc.: 77.50%


In [71]:
vocab_size1 = vocab_size + 100000

lstm_model = build_rnn_model(
    embedding_dim, vocab_size1,
    recurrent_type='LSTM',
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

lstm_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          5162240   
                                                                 
 bidir-lstm-layer-0 (Bidirec  (None, 128)              43520     
 tional)                                                         
                                                                 
 dense_26 (Dense)            (None, 64)                8256      
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,214,081
Trainable params: 5,214,081
Non-trainable params: 0
_________________________________________________________________


In [72]:
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                   metrics=['accuracy'])

history = lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=2)

Epoch 1/2
625/625 [==============================] - 689s 1s/step - loss: 0.5013 - accuracy: 0.7491 - val_loss: 0.3560 - val_accuracy: 0.8490
Epoch 2/2
625/625 [==============================] - 685s 1s/step - loss: 0.2374 - accuracy: 0.9113 - val_loss: 0.3373 - val_accuracy: 0.8650


In [73]:
results_lstm = lstm_model.evaluate(test_data)

782/782 [==============================] - 171s 218ms/step - loss: 0.3498 - accuracy: 0.8627


In [74]:
print('Test Acc.: {:.2f}%'.format(results_lstm[1]*100))

Test Acc.: 86.27%
